# Using GloVe Embedding

© Data Trainers LLC. GPL v 3.0.

**Author:** Axel Sirota


In this notebook we will leverage Standford's GloVe vectors which is a pretrained embedding on 1.4B Tweets.

Take it easy and pay attention to the main differences with before, and the non-trainable parameters. Finally, check how accurate the results now are.
You can run this lab both locally or in Colab.

- To run in Colab just go to `https://colab.research.google.com`, sign-in and you upload this notebook. Colab has GPU access for free.
- To run locally just run `jupyter notebook` and access the notebook in this lab. You would need to first install the requirements in `requirements.txt`

Follow the instructions. Good luck!


In [ ]:
!nvidia-smi

Mon May 13 08:24:06 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   49C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
!pip install textblob 'keras-nlp' 'keras-preprocessing' 'gensim==4.2.0' 'tensorflow-text==2.15.0' np_utils

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 515.3/515.3 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.0/24.0 MB 29.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 28.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 950.8/950.8 kB 38.7 MB/s eta 0:00:00
  Created wheel for np_utils: filename=np_utils-0.6.0-py3-none-any.whl size=56441 sha256=ac00e9ccd36fb809d5183f7c23d324f542cd1b98484e2d6ffe87e52a55d5a75b
  Stored in directory: /root/.cache/pip/wheels/b6/c7/50/2307607f44366dd021209f660045f8d51cb976514d30be7cc7
Successfully built np_utils
  Attempting uninstall: gensim
    Found existing installation: gensim 4.3.2
    Uninstalling gensim-4.3.2:
      Successfully uninstalled gensim-4.3.2


In [ ]:
import multiprocessing
import tensorflow as tf
import sys
import keras.backend as K
from keras.models import Sequential
from keras.layers import Dense, Embedding, Lambda
import np_utils
from tensorflow.keras.utils import to_categorical
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from textblob import TextBlob, Word
from keras_preprocessing.sequence import pad_sequences
from keras.initializers import Constant
import numpy as np
import random
import os
import pandas as pd
import gensim
import warnings
import nltk

TRACE = False
embedding_dim = 100
epochs=2
batch_size = 64
BATCH = True

def set_seeds_and_trace():
  os.environ['PYTHONHASHSEED'] = '0'
  np.random.seed(42)
  tf.random.set_seed(42)
  random.seed(42)
  if TRACE:
    tf.debugging.set_log_device_placement(True)

def set_session_with_gpus_and_cores():
  cores = multiprocessing.cpu_count()
  gpus = len(tf.config.list_physical_devices('GPU'))
  config = tf.compat.v1.ConfigProto( device_count = {'GPU': gpus  , 'CPU': cores} , intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
  sess = tf.compat.v1.Session(config=config)
  tf.compat.v1.keras.backend.set_session(sess)

set_seeds_and_trace()
set_session_with_gpus_and_cores()
warnings.filterwarnings('ignore')
nltk.download('punkt')
textblob_tokenizer = lambda x: TextBlob(x).words

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
%%writefile get_data.sh
if [ ! -f yelp.csv ]; then
  wget -O yelp.csv https://www.dropbox.com/s/xds4lua69b7okw8/yelp.csv?dl=0
fi

if [ ! -f glove.6B.100d.txt ]; then
  wget -O glove.6B.100d.txt https://www.dropbox.com/s/dl1vswq2sz5f1ws/glove.6B.100d.txt?dl=0
fi

Writing get_data.sh


In [ ]:
!bash get_data.sh

--2024-05-13 08:28:15--  https://www.dropbox.com/s/xds4lua69b7okw8/yelp.csv?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.81.18, 2620:100:6031:18::a27d:5112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.81.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/xds4lua69b7okw8/yelp.csv [following]
--2024-05-13 08:28:15--  https://www.dropbox.com/s/raw/xds4lua69b7okw8/yelp.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc3fe7c7a80f39b065cb7f8b0de5.dl.dropboxusercontent.com/cd/0/inline/CSyoJSvn_MilY09gV8UlukeEdxwAR5mTpnlZ2I0Wm6dig7Z5RxqWjsdROKQgvdterDSrJl-tOl-OKtLVfDgHOuH91jLwwv9ajevJUXoyO8bcM6TxLZCf5riAgRE6Lc-4a-okpD9W8v6lrVoMNZCGaOxO/file# [following]
--2024-05-13 08:28:16--  https://uc3fe7c7a80f39b065cb7f8b0de5.dl.dropboxusercontent.com/cd/0/inline/CSyoJSvn_MilY09gV8UlukeEdxwAR5mTpnlZ2I0Wm6dig7Z5RxqWjsdROKQgvdterDSrJl-tOl-OKtLVfDgHOuH91jLwwv9ajevJUXoyO8

In [ ]:
path = './yelp.csv'
yelp = pd.read_csv(path)
# Create a new DataFrame that only contains the 5-star and 1-star reviews to have extremes.
yelp_best_worst = yelp[(yelp.stars==5) | (yelp.stars==1)]
X = yelp_best_worst.text
y = yelp_best_worst.stars.map({1:0, 5:1})

In [ ]:
corpus = [sentence for sentence in X.values if type(sentence) == str and len(TextBlob(sentence).words) > 3]

In [ ]:
path_to_glove_file = "./glove.6B.100d.txt"
embeddings_index = {}
# Construct a function that fills the embedding_index dict for every word in the GloVe file with its coefficients.
# HELP: For that iterate over the Glove file (hint: check that file to view its structure first!), split the word from the numbers, and populate the dictionary with the word and the numbers as a numpy array.
# Hint2: check np.fromstring
# FILL

print("Found %s word vectors." % len(embeddings_index))

Found 0 word vectors.


In [ ]:
with open(path_to_glove_file) as f:
  lines = f.readlines()

In [ ]:
for line in lines:
  data = line.split()
  embeddings_index[data[0]] = np.array(data[1:])

print("Found %s word vectors." % len(embeddings_index))

Found 400001 word vectors.


In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(corpus)
tokenized_corpus = tokenizer.texts_to_sequences(corpus)
nb_samples = sum(len(s) for s in corpus)
vocab_size = len(tokenizer.word_index) + 1

In [ ]:
print(f'First 5 corpus items are {corpus[:5]}')
print(f'Length of corpus is {len(corpus)}')

First 5 corpus items are ['My wife took me here on my birthday for breakfast and it was excellent.  The weather was perfect which made sitting outside overlooking their grounds an absolute pleasure.  Our waitress was excellent and our food arrived quickly on the semi-busy Saturday morning.  It looked like the place fills up pretty quickly so the earlier you get here the better.\n\nDo yourself a favor and get their Bloody Mary.  It was phenomenal and simply the best I\'ve ever had.  I\'m pretty sure they only use ingredients from their garden and blend them fresh when you order it.  It was amazing.\n\nWhile EVERYTHING on the menu looks excellent, I had the white truffle scrambled eggs vegetable skillet and it was tasty and delicious.  It came with 2 pieces of their griddled bread with was amazing and it absolutely made the meal complete.  It was the best "toast" I\'ve ever had.\n\nAnyway, I can\'t wait to go back!', 'I have no idea why some people give bad reviews about this place. It g

In [ ]:
hits = 0
misses = 0

embedding_matrix = np.zeros((vocab_size, embedding_dim))

# Create a loop such that for every word in the vocabulary, if it exists in the Glove embedding, then set for that word (that means that index) the tensor of the Glove Embedding. Otherwise fill with 0
# FILL

for word, idx in tokenizer.word_index.items():
  glv_embedding = embeddings_index.get(word, None)
  if glv_embedding is not None:
    embedding_matrix[idx] = glv_embedding
    hits += 1
  else:
    misses += 1

print(hits)
print(misses)

# In the end, the embedding matrix should have, for every word of our tokenizer that exists in GloVe, the tensor representation of that word

17190
2759


In [51]:
model = Sequential()
# Add the new embedding and set it as non-trainable (although you could fine-tune it if you prefer)
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, weights=[embedding_matrix], trainable=False))
# Add a Lambda layer to average out the words dimension
model.add(Lambda(lambda x: K.mean(x, axis=1)))
# Add a Dense layer to classify words
model.add(Dense(vocab_size, activation='softmax'))


In [52]:
model.compile(loss='categorical_crossentropy', optimizer='adam')
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, None, 100)         1995000   
                                                                 
 lambda_2 (Lambda)           (None, 100)               0         
                                                                 
 dense_2 (Dense)             (None, 19950)             2014950   
                                                                 
Total params: 4009950 (15.30 MB)
Trainable params: 2014950 (7.69 MB)
Non-trainable params: 1995000 (7.61 MB)
_________________________________________________________________


Notice the Non-trainable parameters! What we are doing is just training the softmax based on correct embeddings. This is called fine tuning the embedding.


In [53]:
# This is the algorithmic part of batching the dataset and yielding the window of words and expected middle word for each bacth as a generator.
def generate_data(corpus, vocab_size, window_size=2, sentence_batch_size=10,  batch_size=250):
    number_of_sentence_batches = (len(corpus) // sentence_batch_size) + 1
    for batch in range(number_of_sentence_batches):
        lower_end = batch*batch_size
        upper_end = (batch+1)*batch_size if batch+1 < number_of_sentence_batches else len(corpus)
        mini_batch_size = upper_end - lower_end
        maxlen = window_size*2
        X = []
        Y = []
        for review_id, words in enumerate(corpus[lower_end:upper_end]):
            L = len(words)
            for index, word in enumerate(words):
                contexts = []
                labels   = []
                s = index - window_size
                e = index + window_size + 1

                contexts.append([words[i] for i in range(s, e) if 0 <= i < L and i != index])
                labels.append(word)

                x = pad_sequences(contexts, maxlen=maxlen)
                y = to_categorical(labels, vocab_size)
                X.append(x)
                Y.append(y)
        X = tf.constant(X)
        Y = tf.constant(Y)
        number_of_batches = len(X) // batch_size
        for real_batch in range(number_of_batches):
          lower_end = real_batch*batch_size
          upper_end = (real_batch+1)*batch_size
          batch_X = tf.squeeze(X[lower_end:upper_end])
          batch_Y = tf.squeeze(Y[lower_end:upper_end])
          yield (batch_X, batch_Y)

In [54]:
iterable = generate_data(corpus=tokenized_corpus[:10], vocab_size=vocab_size, batch_size=64)
sample_x, sample_y = next(iterable)
sample_y_numpy = sample_y.numpy()
sample_x

<tf.Tensor: shape=(64, 4), dtype=int32, numpy=
array([[   0,    0,  447,  202],
       [   0,   12,  202,   35],
       [  12,  447,   35,   41],
       [ 447,  202,   41,   20],
       [ 202,   35,   20,   12],
       [  35,   41,   12,  571],
       [  41,   20,  571,   11],
       [  20,   12,   11,  282],
       [  12,  571,  282,    2],
       [ 571,   11,    2,    9],
       [  11,  282,    9,    8],
       [ 282,    2,    8,  196],
       [   2,    9,  196,    1],
       [   9,    8,    1, 1549],
       [   8,  196, 1549,    8],
       [ 196,    1,    8,  201],
       [   1, 1549,  201,   71],
       [1549,    8,   71,  123],
       [   8,  201,  123,  654],
       [ 201,   71,  654,  319],
       [  71,  123,  319, 4500],
       [ 123,  654, 4500,   43],
       [ 654,  319,   43, 2394],
       [ 319, 4500, 2394,   58],
       [4500,   43,   58, 1408],
       [  43, 2394, 1408, 1478],
       [2394,   58, 1478,   50],
       [  58, 1408,   50,  483],
       [1408, 1478,  483,    

In [55]:
def fit_model():
    if not BATCH:
        # If we are not batching, Fill how to get X AND Y
        X, Y = None # Fill
        print(f'Size of X is {X.shape} and Y is {Y.shape}')
        model.fit(X, Y, epochs = epochs)
    else:
        for epoch in range(epochs):
          batch_count = 0
          for x, y in generate_data(corpus=tokenized_corpus[:100], vocab_size=vocab_size, batch_size=batch_size):
              print(f"Epoch {epoch+1} Batch {batch_count+1}")
              history = model.train_on_batch(x, y, return_dict=True)
              print(f'Loss: {history["loss"]}')
              batch_count += 1
              if batch_count > batch_size:
                break

In [56]:
fit_model()

Epoch 1 Batch 1
Loss: 9.900870323181152
Epoch 1 Batch 2
Loss: 9.882753372192383
Epoch 1 Batch 3
Loss: 9.875391006469727
Epoch 1 Batch 4
Loss: 9.859234809875488
Epoch 1 Batch 5
Loss: 9.850264549255371
Epoch 1 Batch 6
Loss: 9.805156707763672
Epoch 1 Batch 7
Loss: 9.815977096557617
Epoch 1 Batch 8
Loss: 9.828136444091797
Epoch 1 Batch 9
Loss: 9.787300109863281
Epoch 1 Batch 10
Loss: 9.779848098754883
Epoch 1 Batch 11
Loss: 9.730436325073242
Epoch 1 Batch 12
Loss: 9.762035369873047
Epoch 1 Batch 13
Loss: 9.665240287780762
Epoch 1 Batch 14
Loss: 9.739299774169922
Epoch 1 Batch 15
Loss: 9.745723724365234
Epoch 1 Batch 16
Loss: 9.66541862487793
Epoch 1 Batch 17
Loss: 9.54014778137207
Epoch 1 Batch 18
Loss: 9.502429962158203
Epoch 1 Batch 19
Loss: 9.53338623046875
Epoch 1 Batch 20
Loss: 9.642533302307129
Epoch 1 Batch 21
Loss: 9.65206527709961
Epoch 1 Batch 22
Loss: 9.521858215332031
Epoch 1 Batch 23
Loss: 9.465085983276367
Epoch 1 Batch 24
Loss: 9.446490287780762
Epoch 1 Batch 25
Loss: 9.4233

In [57]:
with open('./vectors.txt' ,'w') as f:
    f.write('{} {}\n'.format(vocab_size-1, embedding_dim))
    vectors = model.get_weights()[0]
    for word, i in tokenizer.word_index.items():
        str_vec = ' '.join(map(str, list(vectors[i, :])))
        f.write('{} {}\n'.format(word, str_vec))


In [58]:
w2v = gensim.models.KeyedVectors.load_word2vec_format('./vectors.txt', binary=False)

In [59]:
w2v.most_similar(positive=['pizza'])

[('sandwich', 0.7488481402397156),
 ('sandwiches', 0.7221847772598267),
 ('bread', 0.700667679309845),
 ('pizzas', 0.7004502415657043),
 ('burger', 0.6977595686912537),
 ('snack', 0.6840099692344666),
 ('taco', 0.6788282990455627),
 ('pie', 0.6776915788650513),
 ('chicken', 0.6714670658111572),
 ('burgers', 0.66903156042099)]

In [60]:
w2v.most_similar(positive=['grape'])

[('grapes', 0.7915149927139282),
 ('wine', 0.7132691144943237),
 ('varieties', 0.7115378975868225),
 ('chardonnay', 0.7091464996337891),
 ('wines', 0.7076444029808044),
 ('pinot', 0.7055837512016296),
 ('zinfandel', 0.7001710534095764),
 ('vines', 0.681800127029419),
 ('sauvignon', 0.6581584811210632),
 ('cabernet', 0.6564268469810486)]

Do you notice the difference in the accuracy? For any task first search if there are any pretrained models to use!